In [0]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable
import time

In [0]:
display(dbutils.fs.mkdirs("dbfs:/Bronze"))
display(dbutils.fs.mkdirs("dbfs:/Silver"))
display(dbutils.fs.mkdirs("dbfs:/Gold"))

TrueTrue

In [0]:
display(dbutils.fs.ls("FileStore/tables"))

path,name,size,modificationTime
dbfs:/FileStore/tables/Global_Superstore2.csv,Global_Superstore2.csv,12089916,1743651204000


In [0]:
dbutils.fs.mv("dbfs:/FileStore/tables/Global_Superstore2.csv","dbfs:/Bronze")

Out[14]: True

In [0]:
'''
Ler o CSV original com Spark.
Realizar ajustes de tipo de dados quando necessário (ex: datas, números).
Salvar o dataset em JSON, Parquet, Avro e Delta.
Garantir que o JSON inclua listas aninhadas para pelo menos uma coluna, se aplicável.

'''

In [0]:
df = spark.read.csv("dbfs:/Bronze/Global_Superstore2.csv",header=True,inferSchema=True,sep= ",")
display(df.head(10))

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024,US,East,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wireless Headset System,2309.65,7,0,762.1845,933.57,Critical
26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,null,APAC,Oceania,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.765,923.63,Critical
25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,null,APAC,Oceania,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.971,915.49,Medium
13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,null,EU,Central,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.51,5,0.1,-96.54,910.16,Medium
47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,Senegal,null,Africa,Africa,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.96,8,0,311.52,903.04,Critical
22732,IN-2013-42360,2013-06-28,2013-07-01,Second Class,JM-15655,Jim Mitchum,Corporate,Sydney,New South Wales,Australia,null,APAC,Oceania,TEC-PH-10000030,Technology,Phones,"Samsung Smart Phone, with Caller ID",2862.675,5,0.1,763.275,897.35,Critical
30570,IN-2011-81826,2011-11-07,2011-11-09,First Class,TS-21340,Toby Swindell,Consumer,Porirua,Wellington,New Zealand,null,APAC,Oceania,FUR-CH-10004050,Furniture,Chairs,"Novimex Executive Leather Armchair, Adjustable",1822.08,4,0,564.84,894.77,Critical
31192,IN-2012-86369,2012-04-14,2012-04-18,Standard Class,MB-18085,Mick Brown,Consumer,Hamilton,Waikato,New Zealand,null,APAC,Oceania,FUR-TA-10002958,Furniture,Tables,"Chromcraft Conference Table, Fully Assembled",5244.84,6,0,996.48,878.38,High
40155,CA-2014-135909,2014-10-14,2014-10-21,Standard Class,JW-15220,Jane Waco,Corporate,Sacramento,California,United States,95823,US,West,OFF-BI-10003527,Office Supplies,Binders,Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,5083.96,5,0.2,1906.485,867.69,Low
40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,United States,28027,US,South,FUR-TA-10000198,Furniture,Tables,Chromcraft Bull-Nose Wood Oval Conference Tables & Bases,4297.644,13,0.4,-1862.3124,865.74,Critical


In [0]:
df_sales_to_double = df.withColumn("Sales", col("Sales").cast("double"))
df_qnt_to_int = df_sales_to_double.withColumn("Quantity", col("Quantity").cast("int"))
df_discount_to_double = df_qnt_to_int.withColumn("Discount", col("Discount").cast("double"))

In [0]:
df_discount_to_double.printSchema()


root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Market: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Shipping Cost: double (nullable = true)
 |-- Order Priority: string (nullable = true)



In [0]:
df_discount_to_double.write.format("parquet").mode("overwrite").save("dbfs:/Silver/Global_Superstore2.parquet")

In [0]:
df_silver = spark.read.parquet("dbfs:/Silver/Global_Superstore2.parquet")
display(df_silver.head(10))

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024,US,East,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wireless Headset System,2309.65,7,0.0,762.1845,933.57,Critical
26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,null,APAC,Oceania,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.765,923.63,Critical
25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,null,APAC,Oceania,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.971,915.49,Medium
13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,null,EU,Central,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.51,5,0.1,-96.54,910.16,Medium
47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,Senegal,null,Africa,Africa,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.96,8,0.0,311.52,903.04,Critical
22732,IN-2013-42360,2013-06-28,2013-07-01,Second Class,JM-15655,Jim Mitchum,Corporate,Sydney,New South Wales,Australia,null,APAC,Oceania,TEC-PH-10000030,Technology,Phones,"Samsung Smart Phone, with Caller ID",2862.675,5,0.1,763.275,897.35,Critical
30570,IN-2011-81826,2011-11-07,2011-11-09,First Class,TS-21340,Toby Swindell,Consumer,Porirua,Wellington,New Zealand,null,APAC,Oceania,FUR-CH-10004050,Furniture,Chairs,"Novimex Executive Leather Armchair, Adjustable",1822.08,4,0.0,564.84,894.77,Critical
31192,IN-2012-86369,2012-04-14,2012-04-18,Standard Class,MB-18085,Mick Brown,Consumer,Hamilton,Waikato,New Zealand,null,APAC,Oceania,FUR-TA-10002958,Furniture,Tables,"Chromcraft Conference Table, Fully Assembled",5244.84,6,0.0,996.48,878.38,High
40155,CA-2014-135909,2014-10-14,2014-10-21,Standard Class,JW-15220,Jane Waco,Corporate,Sacramento,California,United States,95823,US,West,OFF-BI-10003527,Office Supplies,Binders,Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,5083.96,5,0.2,1906.485,867.69,Low
40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,United States,28027,US,South,FUR-TA-10000198,Furniture,Tables,Chromcraft Bull-Nose Wood Oval Conference Tables & Bases,4297.644,13,0.4,-1862.3124,865.74,Critical


In [0]:
'''
Adicionar uma coluna lucro_percentual = Profit / Sales.
Garantir que todas as colunas estejam com os tipos corretos (ex: datas como date, valores como double, etc).
Verificar se os tipos foram mantidos corretamente após a conversão para cada formato.

'''

In [0]:
df_silver.dropna(how="all")
df_silver.dropDuplicates()

Out[84]: DataFrame[Row ID: int, Order ID: string, Order Date: date, Ship Date: date, Ship Mode: string, Customer ID: string, Customer Name: string, Segment: string, City: string, State: string, Country: string, Postal Code: int, Market: string, Region: string, Product ID: string, Category: string, Sub-Category: string, Product Name: string, Sales: double, Quantity: int, Discount: double, Profit: double, Shipping Cost: double, Order Priority: string]

In [0]:
df_lucro = df_silver.withColumn("lucro percentual", round(col('Profit') / col('Sales'),2 )) #.cast("decimal(10,2)"))
display(df_lucro.head(10))

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,lucro percentual
32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024,US,East,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wireless Headset System,2309.65,7,0.0,762.1845,933.57,Critical,0.33
26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,null,APAC,Oceania,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.765,923.63,Critical,-0.08
25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,null,APAC,Oceania,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.971,915.49,Medium,0.18
13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,null,EU,Central,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.51,5,0.1,-96.54,910.16,Medium,-0.03
47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,Senegal,null,Africa,Africa,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.96,8,0.0,311.52,903.04,Critical,0.11
22732,IN-2013-42360,2013-06-28,2013-07-01,Second Class,JM-15655,Jim Mitchum,Corporate,Sydney,New South Wales,Australia,null,APAC,Oceania,TEC-PH-10000030,Technology,Phones,"Samsung Smart Phone, with Caller ID",2862.675,5,0.1,763.275,897.35,Critical,0.27
30570,IN-2011-81826,2011-11-07,2011-11-09,First Class,TS-21340,Toby Swindell,Consumer,Porirua,Wellington,New Zealand,null,APAC,Oceania,FUR-CH-10004050,Furniture,Chairs,"Novimex Executive Leather Armchair, Adjustable",1822.08,4,0.0,564.84,894.77,Critical,0.31
31192,IN-2012-86369,2012-04-14,2012-04-18,Standard Class,MB-18085,Mick Brown,Consumer,Hamilton,Waikato,New Zealand,null,APAC,Oceania,FUR-TA-10002958,Furniture,Tables,"Chromcraft Conference Table, Fully Assembled",5244.84,6,0.0,996.48,878.38,High,0.19
40155,CA-2014-135909,2014-10-14,2014-10-21,Standard Class,JW-15220,Jane Waco,Corporate,Sacramento,California,United States,95823,US,West,OFF-BI-10003527,Office Supplies,Binders,Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,5083.96,5,0.2,1906.485,867.69,Low,0.38
40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,United States,28027,US,South,FUR-TA-10000198,Furniture,Tables,Chromcraft Bull-Nose Wood Oval Conference Tables & Bases,4297.644,13,0.4,-1862.3124,865.74,Critical,-0.43


In [0]:
df_lucro.printSchema

Out[7]: <bound method DataFrame.printSchema of DataFrame[Row ID: int, Order ID: string, Order Date: date, Ship Date: date, Ship Mode: string, Customer ID: string, Customer Name: string, Segment: string, City: string, State: string, Country: string, Postal Code: int, Market: string, Region: string, Product ID: string, Category: string, Sub-Category: string, Product Name: string, Sales: double, Quantity: int, Discount: double, Profit: double, Shipping Cost: double, Order Priority: string, lucro_percentual: double]>

In [0]:
'''
Extrair o ano da coluna Order Date.
Escrever os dados particionados em:
Parquet por Country
Delta por Order Year
Verificar os diretórios gerados no DBFS.

'''

Out[60]: '\nExtrair o ano da coluna Order Date.\nEscrever os dados particionados em:\nParquet por Country\nDelta por Order Year\nVerificar os diretórios gerados no DBFS.\n\n'

In [0]:
df_order_date = df_lucro.withColumn("YearOrderDate", year(col("Order Date")))
display(df_order_date.head(10))

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,lucro percentual,YearOrderDate
32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024,US,East,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wireless Headset System,2309.65,7,0.0,762.1845,933.57,Critical,0.33,2012
26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,null,APAC,Oceania,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.765,923.63,Critical,-0.08,2013
25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,null,APAC,Oceania,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.971,915.49,Medium,0.18,2013
13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,null,EU,Central,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.51,5,0.1,-96.54,910.16,Medium,-0.03,2013
47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,Senegal,null,Africa,Africa,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.96,8,0.0,311.52,903.04,Critical,0.11,2013
22732,IN-2013-42360,2013-06-28,2013-07-01,Second Class,JM-15655,Jim Mitchum,Corporate,Sydney,New South Wales,Australia,null,APAC,Oceania,TEC-PH-10000030,Technology,Phones,"Samsung Smart Phone, with Caller ID",2862.675,5,0.1,763.275,897.35,Critical,0.27,2013
30570,IN-2011-81826,2011-11-07,2011-11-09,First Class,TS-21340,Toby Swindell,Consumer,Porirua,Wellington,New Zealand,null,APAC,Oceania,FUR-CH-10004050,Furniture,Chairs,"Novimex Executive Leather Armchair, Adjustable",1822.08,4,0.0,564.84,894.77,Critical,0.31,2011
31192,IN-2012-86369,2012-04-14,2012-04-18,Standard Class,MB-18085,Mick Brown,Consumer,Hamilton,Waikato,New Zealand,null,APAC,Oceania,FUR-TA-10002958,Furniture,Tables,"Chromcraft Conference Table, Fully Assembled",5244.84,6,0.0,996.48,878.38,High,0.19,2012
40155,CA-2014-135909,2014-10-14,2014-10-21,Standard Class,JW-15220,Jane Waco,Corporate,Sacramento,California,United States,95823,US,West,OFF-BI-10003527,Office Supplies,Binders,Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,5083.96,5,0.2,1906.485,867.69,Low,0.38,2014
40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,United States,28027,US,South,FUR-TA-10000198,Furniture,Tables,Chromcraft Bull-Nose Wood Oval Conference Tables & Bases,4297.644,13,0.4,-1862.3124,865.74,Critical,-0.43,2012


In [0]:
df_order_date.write.format("parquet").mode("overwrite").partitionBy("Country").save("dbfs:/Silver/Country.parquet")

In [0]:
new_column_names = []
for column in df_order_date.columns:
    new_column_names.append(column.replace(" ", "").replace("-", ""))
df_order_date = df_order_date.toDF(*new_column_names)

print(df_order_date.columns)


['RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID', 'CustomerName', 'Segment', 'City', 'State', 'Country', 'PostalCode', 'Market', 'Region', 'ProductID', 'Category', 'SubCategory', 'ProductName', 'Sales', 'Quantity', 'Discount', 'Profit', 'ShippingCost', 'OrderPriority', 'lucropercentual', 'YearOrderDate']


In [0]:
df_order_date.write.format("delta").mode("overwrite").partitionBy("YearOrderDate").save("dbfs:/Silver/Year_Order_Date.delta")

In [0]:
df_delta_read = spark.read.format("delta").load("dbfs:/Silver/Year_Order_Date.delta")
display(df_delta_read.head(10))

RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,City,State,Country,PostalCode,Market,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,ShippingCost,OrderPriority,lucropercentual,YearOrderDate
30570,IN-2011-81826,2011-11-07,2011-11-09,First Class,TS-21340,Toby Swindell,Consumer,Porirua,Wellington,New Zealand,null,APAC,Oceania,FUR-CH-10004050,Furniture,Chairs,"Novimex Executive Leather Armchair, Adjustable",2425.1884800000003,4,0.0,564.84,894.77,Critical,0.31,2011
34577,CA-2011-102988,2011-04-05,2011-04-09,Second Class,GM-14695,Greg Maxwell,Corporate,Alexandria,Virginia,United States,22304,US,South,OFF-SU-10002881,Office Supplies,Supplies,Martin Yale Chadless Opener Electric Letter Opener,4164.05,5,0.0,83.281,846.54,High,0.02,2011
45794,SA-2011-1830,2011-12-27,2011-12-29,Second Class,MM-7260,Magdelene Morse,Consumer,Jizan,Jizan,Saudi Arabia,null,EMEA,EMEA,TEC-CIS-10001717,Technology,Phones,"Cisco Smart Phone, with Caller ID",2616.96,4,0.0,1151.4,832.41,Critical,0.44,2011
31784,CA-2011-154627,2011-10-29,2011-10-31,First Class,SA-20830,Sue Ann Reed,Consumer,Chicago,Illinois,United States,60610,US,Central,TEC-PH-10001363,Technology,Phones,Apple iPhone 5S,2735.952,6,0.2,341.994,752.51,High,0.13,2011
21586,IN-2011-44803,2011-05-02,2011-05-03,First Class,JK-15325,Jason Klamczynski,Corporate,Suzhou,Anhui,China,null,APAC,North Asia,FUR-CH-10000027,Furniture,Chairs,"SAFCO Executive Leather Armchair, Black",2754.0,6,0.0,358.02,752.47,Critical,0.13,2011
30191,IN-2011-10286,2011-12-17,2011-12-20,First Class,PB-19210,Phillip Breyer,Corporate,Taipei,Taipei City,Taiwan,null,APAC,North Asia,FUR-TA-10004744,Furniture,Tables,"Lesro Conference Table, with Bottom Storage",1715.16,2,0.0,720.36,725.57,Critical,0.42,2011
11645,ES-2011-4699764,2011-03-14,2011-03-17,Second Class,EB-14110,Eugene Barchas,Consumer,Leipzig,Saxony,Germany,null,EU,Central,OFF-AP-10004512,Office Supplies,Appliances,"Hoover Stove, Red",3069.738,6,0.1,1364.238,725.34,Critical,0.44,2011
220,US-2011-128776,2011-12-28,2011-12-30,Second Class,RR-19525,Rick Reed,Corporate,Santo Domingo,Santo Domingo,Dominican Republic,null,LATAM,Caribbean,TEC-PH-10002815,Technology,Phones,"Samsung Smart Phone, VoIP",1696.64,5,0.2,-148.46,704.06,Critical,-0.09,2011
21286,IN-2011-28087,2011-11-03,2011-11-05,Second Class,DP-13105,Dave Poirier,Corporate,Gold Coast,Queensland,Australia,null,APAC,Oceania,OFF-AP-10004246,Office Supplies,Appliances,"Breville Stove, Red",2526.93,5,0.1,561.48,689.8,Critical,0.22,2011
32543,CA-2011-168494,2011-12-12,2011-12-14,Second Class,NP-18700,Nora Preis,Consumer,Fresno,California,United States,93727,US,West,FUR-TA-10003473,Furniture,Tables,Bretford Rectangular Conference Table Tops,3610.848,12,0.2,135.4068,683.12,High,0.04,2011


In [0]:
for coluna in df_delta_read.columns:
    df_null = df_delta_read.where(df_delta_read[coluna].isNull()).count()
    display(df_null)

000000000004129600000030020110003000

In [0]:
display(dbutils.fs.ls("dbfs:/Silver/Country.parquet"))

path,name,size,modificationTime
dbfs:/Silver/Country.parquet/Country=Afghanistan/,Country=Afghanistan/,0,0
dbfs:/Silver/Country.parquet/Country=Albania/,Country=Albania/,0,0
dbfs:/Silver/Country.parquet/Country=Algeria/,Country=Algeria/,0,0
dbfs:/Silver/Country.parquet/Country=Angola/,Country=Angola/,0,0
dbfs:/Silver/Country.parquet/Country=Argentina/,Country=Argentina/,0,0
dbfs:/Silver/Country.parquet/Country=Armenia/,Country=Armenia/,0,0
dbfs:/Silver/Country.parquet/Country=Australia/,Country=Australia/,0,0
dbfs:/Silver/Country.parquet/Country=Austria/,Country=Austria/,0,0
dbfs:/Silver/Country.parquet/Country=Azerbaijan/,Country=Azerbaijan/,0,0
dbfs:/Silver/Country.parquet/Country=Bahrain/,Country=Bahrain/,0,0


In [0]:
display(dbutils.fs.ls("dbfs:/Silver/Year_Order_Date.delta/")) 

path,name,size,modificationTime
dbfs:/Silver/Year_Order_Date.delta/YearOrderDate=2011/,YearOrderDate=2011/,0,0
dbfs:/Silver/Year_Order_Date.delta/YearOrderDate=2012/,YearOrderDate=2012/,0,0
dbfs:/Silver/Year_Order_Date.delta/YearOrderDate=2013/,YearOrderDate=2013/,0,0
dbfs:/Silver/Year_Order_Date.delta/YearOrderDate=2014/,YearOrderDate=2014/,0,0
dbfs:/Silver/Year_Order_Date.delta/_delta_log/,_delta_log/,0,0


In [0]:
'''
Fazer uma modificação nos dados Delta (ex: aumentar 10% nas vendas de um país específico).
Listar o histórico de versões com DeltaTable.history().
Ler a versão anterior dos dados usando versionAsOf.

'''

In [0]:
df_delta_table = DeltaTable.forPath(spark, "dbfs:/Silver/Year_Order_Date.delta")

In [0]:
df_delta_table.update(condition="Country = 'New Zealand'", set={"Sales": 'Sales * 1.10'}) #antes:1822.08 depois:2004.288

In [0]:
display(df_delta_table.history())

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-04-03T13:01:29.000+0000,1159263163929264,kethylle.serafim@tech.math.group,UPDATE,"Map(predicate -> [""(Country#5308 = New Zealand)""])",null,List(331819175347738),0403-122229-sycy2n2l,0,WriteSerializable,false,"Map(numRemovedFiles -> 12, numRemovedBytes -> 3334687, numCopiedRows -> 50662, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 8189, scanTimeMs -> 3013, numAddedFiles -> 8, numUpdatedRows -> 628, numAddedBytes -> 3059101, rewriteTimeMs -> 5149)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-04-03T12:53:24.000+0000,1159263163929264,kethylle.serafim@tech.math.group,WRITE,"Map(mode -> Overwrite, partitionBy -> [""YearOrderDate""])",null,List(331819175347738),0403-122229-sycy2n2l,null,WriteSerializable,false,"Map(numFiles -> 12, numOutputRows -> 51290, numOutputBytes -> 3334687)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
df_old_delta_table = spark.read.format("delta").option("versionAsOf", 0).load("dbfs:/Silver/Year_Order_Date.delta")
display(df_old_delta_table.head(10))

RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,City,State,Country,PostalCode,Market,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,ShippingCost,OrderPriority,lucropercentual,YearOrderDate
30570,IN-2011-81826,2011-11-07,2011-11-09,First Class,TS-21340,Toby Swindell,Consumer,Porirua,Wellington,New Zealand,null,APAC,Oceania,FUR-CH-10004050,Furniture,Chairs,"Novimex Executive Leather Armchair, Adjustable",1822.08,4,0.0,564.84,894.77,Critical,0.31,2011
34577,CA-2011-102988,2011-04-05,2011-04-09,Second Class,GM-14695,Greg Maxwell,Corporate,Alexandria,Virginia,United States,22304,US,South,OFF-SU-10002881,Office Supplies,Supplies,Martin Yale Chadless Opener Electric Letter Opener,4164.05,5,0.0,83.281,846.54,High,0.02,2011
45794,SA-2011-1830,2011-12-27,2011-12-29,Second Class,MM-7260,Magdelene Morse,Consumer,Jizan,Jizan,Saudi Arabia,null,EMEA,EMEA,TEC-CIS-10001717,Technology,Phones,"Cisco Smart Phone, with Caller ID",2616.96,4,0.0,1151.4,832.41,Critical,0.44,2011
31784,CA-2011-154627,2011-10-29,2011-10-31,First Class,SA-20830,Sue Ann Reed,Consumer,Chicago,Illinois,United States,60610,US,Central,TEC-PH-10001363,Technology,Phones,Apple iPhone 5S,2735.952,6,0.2,341.994,752.51,High,0.13,2011
21586,IN-2011-44803,2011-05-02,2011-05-03,First Class,JK-15325,Jason Klamczynski,Corporate,Suzhou,Anhui,China,null,APAC,North Asia,FUR-CH-10000027,Furniture,Chairs,"SAFCO Executive Leather Armchair, Black",2754.0,6,0.0,358.02,752.47,Critical,0.13,2011
30191,IN-2011-10286,2011-12-17,2011-12-20,First Class,PB-19210,Phillip Breyer,Corporate,Taipei,Taipei City,Taiwan,null,APAC,North Asia,FUR-TA-10004744,Furniture,Tables,"Lesro Conference Table, with Bottom Storage",1715.16,2,0.0,720.36,725.57,Critical,0.42,2011
11645,ES-2011-4699764,2011-03-14,2011-03-17,Second Class,EB-14110,Eugene Barchas,Consumer,Leipzig,Saxony,Germany,null,EU,Central,OFF-AP-10004512,Office Supplies,Appliances,"Hoover Stove, Red",3069.738,6,0.1,1364.238,725.34,Critical,0.44,2011
220,US-2011-128776,2011-12-28,2011-12-30,Second Class,RR-19525,Rick Reed,Corporate,Santo Domingo,Santo Domingo,Dominican Republic,null,LATAM,Caribbean,TEC-PH-10002815,Technology,Phones,"Samsung Smart Phone, VoIP",1696.64,5,0.2,-148.46,704.06,Critical,-0.09,2011
21286,IN-2011-28087,2011-11-03,2011-11-05,Second Class,DP-13105,Dave Poirier,Corporate,Gold Coast,Queensland,Australia,null,APAC,Oceania,OFF-AP-10004246,Office Supplies,Appliances,"Breville Stove, Red",2526.93,5,0.1,561.48,689.8,Critical,0.22,2011
32543,CA-2011-168494,2011-12-12,2011-12-14,Second Class,NP-18700,Nora Preis,Consumer,Fresno,California,United States,93727,US,West,FUR-TA-10003473,Furniture,Tables,Bretford Rectangular Conference Table Tops,3610.848,12,0.2,135.4068,683.12,High,0.04,2011


In [0]:
'''
Medir e registrar o tempo de leitura dos arquivos nos formatos CSV, Parquet e Delta.
Fazer uma leitura com .count() para forçar a materialização.
Comparar os tempos e tirar conclusões sobre performance de leitura.

'''

In [0]:
df_csv = "dbfs:/Bronze/Global_Superstore2.csv"
df_parquet = "dbfs:/Silver/Global_Superstore2.parquet"
df_delta = "dbfs:/Silver/Year_Order_Date.delta"

def describe_time(format_type ,path):
    start = time.time()
    df_time = spark.read.format(format_type).load(path)
    df_count = df_time.count()
    end= time.time() 
    time_result = end - start 
    display(f"time de leitura para os formatos {format_type}:{time_result:.2f} segundos")
    return time_result

describe_time("csv", df_csv)
describe_time("parquet", df_parquet)
describe_time("delta", df_delta)

'time de leitura para os formatos csv:1.34 segundos''time de leitura para os formatos parquet:1.00 segundos''time de leitura para os formatos delta:0.75 segundos'Out[76]: 0.7539637088775635

In [0]:
'''
csv- foi o formato de arquivo mais lento pra leitura, acredito que por não ser binário e é interpretado como texto
parquet- foi mais rápido por ser binário e ter um formato colunar que diminui o processamento na consulta por ignorar as outras colunas e só trazer as consultas necessárias.
delta- foi ainda mais rápido por ser no formato parquet, herdando sua estrutura colunar e compactação.

'''

Out[80]: '\ncsv- foi o formato de arquivo mais lento pra leitura, acredito que por não ser binário e é interpretado como texto\nparquet- foi mais rápido por ser binário e ter um formato colunar que diminui o processamento na consulta por ignorar as outras colunas e só trazer as consultas necessárias.\ndelta- foi ainda mais rápido por ser no formato parquet, herdando sua estrutura colunar e compactação.\n\n'

In [0]:
df_old_delta_table.write.format("delta").mode("overwrite").save("dbfs:/Gold/Global_Superstore_gold") #disponibilizar na camada gold para futuras análises.